In [ ]:
import Pkg
Pkg.add("Flux")
using CSV
using DataFrames
using Statistics
using Random
using Flux
using Flux: onehotbatch, onecold, logitcrossentropy
using StatsBase
using Plots
using LinearAlgebra

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed ZygoteRules ───────── v0.2.7
   Installed RealDot ───────────── v0.1.0
   Installed PrettyPrint ───────── v0.2.0
   Installed ContextVariablesX ─── v0.1.3
   Installed ShowCases ─────────── v0.1.0
   Installed Accessors ─────────── v0.1.42
   Installed MLUtils ───────────── v0.4.8
   Installed InitialValues ─────── v0.3.1
   Installed IRTools ───────────── v0.4.15
   Installed FLoopsBase ────────── v0.1.1
   Installed MLCore ────────────── v1.0.0
   Installed MicroCollections ──── v0.2.0
   Installed OneHotArrays ──────── v0.2.10
   Installed Zygote ────────────── v0.7.10
   Installed MLStyle ───────────── v0.4.17
   Installed ProgressLogging ───── v0.1.5
   Installed DefineSingletons ──── v0.1.2
   Installed NameResolution ────── v0.1.5
   Installed ChainRules ────────── v1.72.5
   Installed BangBang ──────────── v0.4.4
   Installed CompositionsBase ──── v0.1.2
   Installed FLoops

LoadError: ArgumentError: Package StatsBase not found in current path.
- Run `import Pkg; Pkg.add("StatsBase")` to install the StatsBase package.

In [2]:
import Pkg
Pkg.add("Flux")
using Flux
using DataFrames
using CSV
using Statistics
using Random

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed ZygoteRules ───────── v0.2.7
   Installed RealDot ───────────── v0.1.0
   Installed PrettyPrint ───────── v0.2.0
   Installed Accessors ─────────── v0.1.42
   Installed ContextVariablesX ─── v0.1.3
   Installed InitialValues ─────── v0.3.1
   Installed ShowCases ─────────── v0.1.0
   Installed MLUtils ───────────── v0.4.8
   Installed DefineSingletons ──── v0.1.2
   Installed IRTools ───────────── v0.4.15
   Installed ProgressLogging ───── v0.1.5
   Installed FLoopsBase ────────── v0.1.1
   Installed MLCore ────────────── v1.0.0
   Installed MicroCollections ──── v0.2.0
   Installed OneHotArrays ──────── v0.2.10
   Installed Zygote ────────────── v0.7.10
   Installed CompositionsBase ──── v0.1.2
   Installed JuliaVariables ────── v0.2.4
   Installed NameResolution ────── v0.1.5
   Installed BangBang ──────────── v0.4.4
   Installed ChainRules ────────── v1.72.5
   Installed FLoops 

In [12]:
using Pkg
Pkg.update()
Pkg.add(["Flux", "DataFrames", "CSV", "Statistics"])

    Updating registry at `~/.julia/registries/General.toml`
   Installed Enzyme_jll ─ v0.0.185+0
   Installed LuxLib ───── v1.10.0
   Installed Enzyme ───── v0.13.59
   Installed Lux ──────── v1.16.0
    Updating `~/.julia/environments/v1.10/Project.toml`
  [b2108857] ↑ Lux v1.15.0 ⇒ v1.16.0
    Updating `~/.julia/environments/v1.10/Manifest.toml`
  [7da242da] ↑ Enzyme v0.13.57 ⇒ v0.13.59
  [b2108857] ↑ Lux v1.15.0 ⇒ v1.16.0
  [82251201] ↑ LuxLib v1.9.0 ⇒ v1.10.0
  [7cc45869] ↑ Enzyme_jll v0.0.184+0 ⇒ v0.0.185+0
Precompiling packages...
   1657.6 ms  ✓ Enzyme_jll
   7920.7 ms  ✓ LuxLib
  17885.4 ms  ✓ Lux
   5688.2 ms  ✓ Lux → LuxMLUtilsExt
   7261.4 ms  ✓ Lux → LuxZygoteExt
   6520.8 ms  ✓ Lux → LuxFluxExt
 140992.4 ms  ✓ Enzyme
   4002.0 ms  ✓ Enzyme → EnzymeChainRulesCoreExt
   4194.4 ms  ✓ Enzyme → EnzymeLogExpFunctionsExt
   4683.2 ms  ✓ Enzyme → EnzymeBFloat16sExt
   4709.0 ms  ✓ Enzyme → EnzymeGPUArraysCoreExt
   4287.9 ms  ✓ QuadGK → QuadGKEnzymeExt
   4648.6 ms  ✓ Enzyme → Enz

In [19]:
using Flux
using DataFrames
using CSV
using Statistics
using Random

# Set random seed for reproducibility
Random.seed!(123)

TaskLocalRNG()

In [20]:
# Load the dataset
df = CSV.read("card fraud.csv", DataFrame)

# Verify dataset size
println("Dataset has $(nrow(df)) rows, columns: $(names(df))")

# Select a few numerical features and the target
features = [:AMT_INCOME_TOTAL, :AMT_CREDIT, :AMT_ANNUITY, :DAYS_BIRTH]
data = select(df, vcat(:TARGET, features))

# Replace missing values with column means for features
for col in features
    if any(ismissing, data[!, col])
        mean_val = mean(skipmissing(data[!, col]))
        data[!, col] = coalesce.(data[!, col], mean_val)
    end
end

# Handle TARGET column: ensure no missing values and convert to Float32
if any(ismissing, data[!, :TARGET])
    error("TARGET column contains missing values. Please handle them (e.g., remove rows or impute).")
end
if !(eltype(data[!, :TARGET]) <: Real)
    error("TARGET column must contain numeric values (0 or 1). Found type: $(eltype(data[!, :TARGET]))")
end
y = Float32.(data[!, :TARGET])  # Ensure y is a Float32 vector with 518 elements
X = Matrix{Float32}(data[!, features])  # Feature matrix

# Normalize features to [0, 1]
for i in 1:size(X, 2)
    min_val = minimum(X[:, i])
    max_val = maximum(X[:, i])
    if max_val != min_val
        X[:, i] = (X[:, i] .- min_val) / (max_val - min_val)
    else
        X[:, i] .= 0.0f0
    end
end

# Debug: Print sizes to verify
println("X size: $(size(X))")  # Should be (518, 4)
println("y size: $(length(y))")  # Should be 518

Dataset has 518 rows, columns: ["SK_ID_CURR", "TARGET", "NAME_CONTRACT_TYPE", "CODE_GENDER", "FLAG_OWN_CAR", "FLAG_OWN_REALTY", "CNT_CHILDREN", "AMT_INCOME_TOTAL", "AMT_CREDIT", "AMT_ANNUITY", "AMT_GOODS_PRICE", "NAME_TYPE_SUITE", "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS", "NAME_HOUSING_TYPE", "REGION_POPULATION_RELATIVE", "DAYS_BIRTH", "DAYS_EMPLOYED", "DAYS_REGISTRATION", "DAYS_ID_PUBLISH", "OWN_CAR_AGE", "FLAG_MOBIL", "FLAG_EMP_PHONE", "FLAG_WORK_PHONE", "FLAG_CONT_MOBILE", "FLAG_PHONE", "FLAG_EMAIL", "OCCUPATION_TYPE", "CNT_FAM_MEMBERS", "REGION_RATING_CLIENT", "REGION_RATING_CLIENT_W_CITY", "WEEKDAY_APPR_PROCESS_START", "HOUR_APPR_PROCESS_START", "REG_REGION_NOT_LIVE_REGION", "REG_REGION_NOT_WORK_REGION", "LIVE_REGION_NOT_WORK_REGION", "REG_CITY_NOT_LIVE_CITY", "REG_CITY_NOT_WORK_CITY", "LIVE_CITY_NOT_WORK_CITY", "ORGANIZATION_TYPE", "EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3", "APARTMENTS_AVG", "BASEMENTAREA_AVG", "YEARS_BEGINEXPLUATATION_AVG", "YEARS_BUI

In [21]:
# Check dataset size
num_rows = size(X, 1)
println("Dataset has $num_rows rows")

# Create sequences (10 rows per sequence) only if dataset is large enough
sequence_length = 10
num_features = length(features)
sequences = []
labels = []

if num_rows < sequence_length
    error("Dataset too small: need at least $sequence_length rows, but got $num_rows")
else
    for i in 1:(num_rows - sequence_length + 1)
        push!(sequences, X[i:(i+sequence_length-1), :])
        push!(labels, y[i+sequence_length-1])
    end
end

# Convert to 3D array: (num_features, sequence_length, num_sequences)
X_seq = cat(sequences..., dims=3)  # Initially (sequence_length, num_features, num_sequences)
X_seq = permutedims(X_seq, (2, 1, 3))  # Reshape to (num_features, sequence_length, num_sequences)
y_seq = Float32.(labels)

# Verify sequence creation
println("Created $(length(sequences)) sequences")

Dataset has 518 rows
Created 509 sequences


In [22]:
# Split into training and testing sets
train_size = Int(floor(length(labels) * 0.8))
X_train = X_seq[:, :, 1:train_size]
y_train = y_seq[1:train_size]
X_test = X_seq[:, :, (train_size+1):end]
y_test = y_seq[(train_size+1):end]

102-element Vector{Float32}:
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [23]:
# Define simple LSTM model
model = Chain(
    LSTM(num_features => 32),  # LSTM with 4 input features, 32 hidden units
    Dense(32 => 1, sigmoid)    # Output for binary classification
)

Chain(
  LSTM(4 => 32),                        # 4_736 parameters
  Dense(32 => 1, σ),                    # 33 parameters
)                   # Total: 5 arrays, 4_769 parameters, 18.965 KiB.

In [24]:
# Loss function
loss(x, y) = Flux.binarycrossentropy(model(x), y)

# Optimizer
opt = Flux.setup(ADAM(0.01), model)

(layers = ((cell = (Wi = Leaf(Adam(eta=0.01, beta=(0.9, 0.999), epsilon=1.0e-8), (Float32[0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; … ; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0], Float32[0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; … ; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0], (0.9, 0.999))), Wh = Leaf(Adam(eta=0.01, beta=(0.9, 0.999), epsilon=1.0e-8), (Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], (0.9, 0.999))), bias = Leaf(Adam(eta=0.01, beta=(0.9, 0.999), epsilon=1.0e-8), (Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], (0.9, 0.999)))),), (weight = Leaf(Adam(eta=0.01, beta=(0.9, 0.999), epsilon=1.0e-8), (Float32[0.0 0.0 … 0.0 0.0], Float32[0.0 0.0 … 0.0 0.0], (0.9, 0.999))), bias = Leaf(Adam(eta=0.01, beta=(0.9, 

In [28]:
# Training loop
for epoch in 1:20
    println("Epoch $epoch")
    Flux.reset!(model)  # Reset LSTM state
    for i in 1:size(X_train, 3)
        x = X_train[:, :, i:i]  # Shape: (num_features, sequence_length, 1) = (4, 10, 1)
        y = y_train[i:i]  # Shape: (1,)
        loss_val, grads = Flux.withgradient(model) do m
            # CHANGED: Select last time step's output to match y's shape
            ŷ = m(x)[:, end, :]  # Shape: (1, 1) -> reshape to (1,)
            Flux.binarycrossentropy(vec(ŷ), y)
        end
        Flux.update!(opt, model, grads[1])
    end
end

Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20


In [31]:
# Evaluate model
Flux.reset!(model)
# CHANGED: Select last time step's output for predictions
test_pred = model(X_test)[:, end, :][:] .> 0.5  # Shape: (1, num_test_sequences) -> (num_test_sequences,)
accuracy = mean(test_pred .== y_test)
println("Test accuracy: $accuracy")

Test accuracy: 0.9411764705882353
